Datapack Edition
================

Repovizz2 can be used to edit datapacks previously [uploaded](Upload.ipynb) by yourself or other users.

We will use here the [repovizz2 client library](Authentication.ipynb) to ease the authentication, but it is basically a standard OAuth2 authentication.


In [2]:
from repovizz2 import RepoVizzClient

We will be using those modules:

In [3]:
import json
# This is for the notebook
from IPython.display import IFrame, display

Now, some global parameters to connect to repovizz2

In [4]:
CLIENT_ID = "4991097c-9a0c-4a56-a337-2b2177c5aa74"
CLIENT_SECRET = "97c055e7-5dd1-49e8-a6da-126d58f07ce1"

Repovizz2 authentication
-----------------------

We authenticate the client (this script) against repoVizz2 using a registered user. For more details on the authentication system see [repoVizz2 authentication](Authentication.ipynb).

This will create an iframe in the notebook to authenticate the client, but you can open a new browser to do so, if you want

In [5]:
repovizz_client = RepoVizzClient(client_id=CLIENT_ID,client_secret=CLIENT_SECRET, repovizz_url="http://localhost:8080")
authorization_url = repovizz_client.start_auth()
repovizz_client.start_server(async=True)

# print authorization_url # and click on it
# - or -
# import webbrowser
# webbrowser.open(authorization_url)
# - or -
display(IFrame(authorization_url,800,600))

In [6]:
repovizz_client.finish_auth()

Getting User basic information
------------------------------

We first get info about our identity. We can use /api/v1.0/user to get json information about the current logged user.

In [7]:
myself = repovizz_client.get("/api/v1.0/user")
username = myself['username']
userid = myself['id']

Get a datapack, to be edited
----------------------------

In [24]:
datapack_id = myself['datapacks'][0] #datapack id to retrieve
datapack = repovizz_client.get('/api/v1.0/datapacks/{}'.format(datapack_id))

In [12]:
print datapack.keys()

[u'name', u'owner', u'folder', u'id', u'structure', u'group_edit_permissions']


Editing the Datapack
--------------------

To change a property, just post a dictionary with the modified elements

In [15]:
repovizz_client.post(
    "/api/v1.0/datapacks/{}".format(datapack_id),
    json={
        'name': "New datapack Name",
    },
)

{u'result': u'OK'}

In [16]:
repovizz_client.get('/api/v1.0/datapacks/{}'.format(datapack_id))['name']

u'New datapack Name'

To modify the document (`structure`) you need to resubmit it all again

In [25]:
document = datapack['structure']
print document['children'][0]['text']

Handheld camera


In [26]:
document['children'][0]['text'] = "Camera"
repovizz_client.post(
    "/api/v1.0/datapacks/{}".format(datapack_id),
    json={
        'structure': document,
    },
)

{u'result': u'OK'}

In [27]:
repovizz_client.get('/api/v1.0/datapacks/{}'.format(datapack_id))['structure']['children'][0]['text']

u'Camera'

Adding a Data File
------------------

Let's modify the datapack by adding a new data file representing another sensor

In [29]:
datapack = repovizz_client.get('/api/v1.0/datapacks/{}'.format(datapack_id))
document = datapack['structure']

In [36]:
document['children'][2]['children'].append({
   u'class': u'data',
   u'link': u'emg 2.json',
   u'name': u'EMG 2',
   u'text': u'8-channel EMG data from the MYO armband, 2',
   u'type': u'0746bb48-9b92-4cd3-8d9e-3e1de13269eb'})

In [37]:
document['children'][0]['text'] = "Camera"
repovizz_client.post(
    "/api/v1.0/datapacks/{}".format(datapack_id),
    json={
        'structure': document,
    },
)

{u'result': u'OK'}

Next, we upload the data file

In [41]:
repovizz_client.get('/api/v1.0/datapacks/{}/content'.format(datapack_id)).keys()

[u'repovizz:9ec009d2ea38b6d600f793424739dcdc75bf8a7c',
 u'emg 2.json',
 u'repovizz:34d6e7871c1e080736f1f3da92059c9795c1ab90',
 u'repovizz:67ed051e3f6b998e7d72b55b84ce65008a30d7e8',
 u'repovizz:a910ba389ca2847b6bf548fadad68ccebeee4f54',
 u'repovizz:0162912010aa5cad7f6d16bb487b4300c7e76942',
 u'repovizz:269011f97e51fce33447b83360a4af263c2101fe']

In [43]:
print repovizz_client.post(
    '/api/v1.0/datapacks/{}/content/{}'.format(datapack_id,'emg 2.json'),
    files={'file':open('MYO_example_datapack/emg 2.json')}
)

{u'id': u'b1b0a4d5546c7af78a52270c78ffd0470f20a6e9'}
